# Support Vector Machines -2

In [1]:
# Importig Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score

import warnings 
warnings.filterwarnings('ignore')

## EDA

In [2]:
salary_train_data = pd.read_csv('SalaryData_Train(1).csv')
salary_test_data = pd.read_csv('SalaryData_Test(1).csv')

In [3]:
salary_train_data

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30158,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30159,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
salary_test_data

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,Private,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
15056,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
15057,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
15058,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [5]:
salary_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [6]:
salary_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [7]:
salary_train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,30161.0,38.438115,13.134830,17.0,28.0,37.0,47.0,90.0
educationno,30161.0,10.121316,2.550037,1.0,9.0,10.0,13.0,16.0
capitalgain,30161.0,1092.044064,7406.466611,0.0,0.0,0.0,0.0,99999.0
capitalloss,30161.0,88.302311,404.121321,0.0,0.0,0.0,0.0,4356.0
hoursperweek,30161.0,40.931269,11.980182,1.0,40.0,40.0,45.0,99.0


In [8]:
salary_test_data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,15060.0,38.768327,13.380676,17.0,28.0,37.0,48.0,90.0
educationno,15060.0,10.112749,2.558727,1.0,9.0,10.0,13.0,16.0
capitalgain,15060.0,1120.301594,7703.181842,0.0,0.0,0.0,0.0,99999.0
capitalloss,15060.0,89.041899,406.283245,0.0,0.0,0.0,0.0,3770.0
hoursperweek,15060.0,40.951594,12.062831,1.0,40.0,40.0,45.0,99.0


In [9]:
# Dropping columns which are not required

salary_train_data= salary_train_data.drop(['education', 'native'],axis = 1)
salary_train_data

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,Salary
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...
30156,27,Private,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,<=50K
30157,40,Private,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,>50K
30158,58,Private,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,<=50K
30159,22,Private,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,<=50K


In [10]:
# Dropping columns which are not required

salary_test_data= salary_test_data.drop(['education', 'native'],axis = 1)
salary_test_data

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,Salary
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,<=50K
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,<=50K
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,>50K
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,>50K
4,34,Private,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,Private,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,<=50K
15056,39,Private,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,<=50K
15057,38,Private,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,<=50K
15058,44,Private,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,<=50K


## Feature Encoding

In [11]:
 from sklearn.preprocessing import LabelEncoder
salary_train_data = salary_train_data.apply(LabelEncoder().fit_transform)
salary_train_data

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,Salary
0,22,5,12,4,0,1,4,1,24,0,39,0
1,33,4,12,2,3,0,4,1,0,0,12,0
2,21,2,8,0,5,1,4,1,0,0,39,0
3,36,2,6,2,5,0,2,1,0,0,39,0
4,11,2,12,2,9,5,2,0,0,0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
30156,10,2,11,2,12,5,4,0,0,0,37,0
30157,23,2,8,2,6,0,4,1,0,0,39,1
30158,41,2,8,6,0,4,4,0,0,0,39,0
30159,5,2,8,4,0,3,4,1,0,0,19,0


In [12]:
salary_test_data = salary_test_data.apply(LabelEncoder().fit_transform)
salary_test_data

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,Salary
0,8,2,6,4,6,3,2,1,0,0,39,0
1,21,2,8,2,4,0,4,1,0,0,49,0
2,11,1,11,2,10,0,4,1,0,0,39,1
3,27,2,9,2,6,0,2,1,87,0,39,1
4,17,2,5,4,7,1,4,1,0,0,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15055,16,2,12,4,9,3,4,1,0,0,39,0
15056,22,2,12,0,9,1,4,0,0,0,35,0
15057,21,2,12,2,9,0,4,1,0,0,49,0
15058,27,2,12,0,0,3,1,1,73,0,39,0


In [13]:
# Splitting data into test data and train data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(salary_train_data,salary_train_data['Salary'], test_size=0.3, random_state=42)

In [14]:
x_train

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,Salary
27112,18,2,8,2,2,0,4,1,0,0,39,1
13212,6,2,8,2,5,0,4,1,0,0,39,0
24111,22,2,12,2,3,5,4,0,0,0,59,1
22501,16,2,8,0,7,4,4,0,0,0,31,0
3872,10,3,8,5,0,1,4,1,0,0,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...
29802,19,2,11,2,3,5,4,0,0,0,36,0
5390,24,4,9,2,6,0,4,1,49,0,39,1
860,20,2,8,2,2,0,4,1,0,0,93,1
15795,39,4,6,2,7,5,4,0,0,0,39,0


In [15]:
x_test

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,Salary
217,11,2,14,4,9,1,4,1,0,0,54,1
24912,24,2,8,2,2,0,1,1,0,0,39,0
17780,36,2,8,2,7,0,4,1,0,0,49,1
12484,14,2,8,2,2,0,4,1,0,0,44,0
8890,17,1,12,3,9,1,4,0,0,0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20990,15,2,9,2,11,2,1,0,0,0,39,0
23327,30,4,9,2,2,0,4,1,0,0,39,1
24639,24,2,12,5,9,1,2,0,117,0,59,1
7738,7,2,8,4,11,1,4,1,0,0,29,0


In [16]:
y_train

27112    1
13212    0
24111    1
22501    0
3872     0
        ..
29802    0
5390     1
860      1
15795    0
23654    0
Name: Salary, Length: 21112, dtype: int32

In [17]:
y_test

217      1
24912    0
17780    1
12484    0
8890     0
        ..
20990    0
23327    1
24639    1
7738     0
3882     0
Name: Salary, Length: 9049, dtype: int32

## Building model with Grid Search CV

In [21]:
clf = SVC()
param_grid = [{'kernel':['rbf'],'gamma':[50,5,10,0.5],'C':[15,14,13,12,11,10,0.1,0.001]}]
gsv = GridSearchCV(clf,param_grid,cv=10)
gsv.fit(x_train,y_train)

KeyboardInterrupt: 

In [22]:
gsv.best_params_ , gsv.best_score_ 

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [23]:
clf = SVC(C= 15, gamma = 50)
clf.fit(x_train , y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)
confusion_matrix(y_test, y_pred)

KeyboardInterrupt: 